In [14]:
import pandas as pd

# Function to load and preprocess datasets
def load_datasets(crops_file, nutrients_file, age_nutrients_file):
    try:
        # Load crops dataset
        crops_df = pd.read_csv(crops_file)
        crops_df.columns = crops_df.columns.str.lower()  # Convert column names to lowercase
        crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)  # Convert string values to lowercase
        print(f"Crops dataset loaded successfully. Shape: {crops_df.shape}")
        
        # Load nutrients dataset
        nutrients_df = pd.read_csv(nutrients_file)
        nutrients_df.columns = nutrients_df.columns.str.lower()  # Convert column names to lowercase
        nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)  # Convert string values to lowercase
        print(f"Nutrients dataset loaded successfully. Shape: {nutrients_df.shape}")
        
        # Load age-wise nutrient consumption dataset
        age_nutrients_df = pd.read_csv(age_nutrients_file)
        age_nutrients_df.columns = age_nutrients_df.columns.str.lower()  # Convert column names to lowercase
        age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)  # Convert string values to lowercase
        print(f"Age-wise nutrients dataset loaded successfully. Shape: {age_nutrients_df.shape}")
        
        return crops_df, nutrients_df, age_nutrients_df
    except Exception as e:
        print(f"Error loading datasets: {e}")
        return None, None, None

# Example usage
crops_file = 'final_karnataka_dataset.csv'
nutrients_file = 'nutrients.csv'
age_nutrients_file = 'age_wise_nutrients_consumtion_final.csv'

crops_df, nutrients_df, age_nutrients_df = load_datasets(crops_file, nutrients_file, age_nutrients_file)


Crops dataset loaded successfully. Shape: (9021, 92)
Nutrients dataset loaded successfully. Shape: (112, 11)
Age-wise nutrients dataset loaded successfully. Shape: (8, 11)


/tmp/ipykernel_292202/2346180448.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)  # Convert string values to lowercase
/tmp/ipykernel_292202/2346180448.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)  # Convert string values to lowercase
/tmp/ipykernel_292202/2346180448.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)  # Convert string values to lowercase


In [15]:
# Function to calculate nutrient availability for a selected district
def calculate_nutrient_availability(district_name, crops_df, nutrients_df):
    # Filter crops data for the selected district
    selected_district_crops = crops_df[crops_df['district_name'] == district_name]
    
    if selected_district_crops.empty:
        print(f"No crop data found for the district: {district_name}")
        return None

    print(f"Crop data for district {district_name} loaded successfully.")

    # Merge with nutrient dataset to calculate nutrient contributions
    district_nutrients = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
    
    # Calculate total nutrients by summing nutrient values for all crops in the district
    nutrient_totals = district_nutrients.iloc[:, 6:].sum()  # Adjust column slicing based on nutrient start index
    print(f"Total nutrients available in district {district_name}:")
    print(nutrient_totals)

    return nutrient_totals

# Example usage
district_name = input("Enter the district name: ")
nutrient_totals = calculate_nutrient_availability(district_name, crops_df, nutrients_df)


Enter the district name: chitradurga
Crop data for district chitradurga loaded successfully.
Total nutrients available in district chitradurga:
production                               13074932.02
adult male crossbred (1000 number)            904.05
adult male indigenous (1000 number)         79879.80
adult female crossbred (1000 number)       111800.85
adult female indigenous (1000 number)       65304.75
                                            ...     
vitamin c                                    4081.20
iron                                         2527.40
calcium                                       126.20
potassium                                  312725.00
magnesium                                   86609.00
Length: 96, dtype: float64


In [19]:
# Function to display column names for each dataset
def list_column_names(datasets, dataset_names):
    for df, name in zip(datasets, dataset_names):
        print(f"Columns in {name}:")
        print(df.columns.tolist())
        print("\n")

# Example usage
datasets = [crops_df, nutrients_df, age_nutrients_df]
dataset_names = ['crops_df', 'nutrients_df', 'age_nutrients_df']

list_column_names(datasets, dataset_names)


Columns in crops_df:
['state_name', 'district_name', 'crop_year', 'season', 'crop', 'area', 'production', 'adult male crossbred (1000 number)', 'adult male indigenous (1000 number)', 'adult female crossbred (1000 number)', 'adult female indigenous (1000 number)', 'young male crossbred (1000 number)', 'young male indigenous (1000 number)', 'young female crossbred (1000 number)', 'young female indigenous (1000 number)', 'actual rainfall', 'normal rainfall', 'rice.area..1000.ha.', 'rice.production..1000.tons.', 'rice.yield..kg.per.ha.', 'wheat.area..1000.ha.', 'wheat.production..1000.tons.', 'wheat.yield..kg.per.ha.', 'kharif.sorghum.area..1000.ha.', 'kharif.sorghum.production..1000.tons.', 'kharif.sorghum.yield..kg.per.ha.', 'rabi.sorghum.area..1000.ha.', 'rabi.sorghum.production..1000.tons.', 'rabi.sorghum.yield..kg.per.ha.', 'sorghum.area..1000.ha.', 'sorghum.production..1000.tons.', 'sorghum.yield..kg.per.ha.', 'pearl.millet.area..1000.ha.', 'pearl.millet.production..1000.tons.', 'pea

In [20]:
# Function to identify nutrient deficiency for a selected district
def identify_nutrient_deficiency(district_name, crops_df, nutrients_df, age_nutrients_df):
    # Standardize column names in age_nutrients_df
    age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip()
    
    # Define nutrient columns to ensure consistency
    nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c', 
                        'iron', 'calcium', 'potassium', 'magnesium']
    
    # Check if all nutrient_columns exist in age_nutrients_df
    missing_columns = [col for col in nutrient_columns if col not in age_nutrients_df.columns]
    if missing_columns:
        raise KeyError(f"The following columns are missing in age_nutrients_df: {missing_columns}")
    
    # Filter crops data for the selected district
    selected_district_crops = crops_df[crops_df['district_name'] == district_name]
    
    if selected_district_crops.empty:
        print(f"No crop data found for the district: {district_name}")
        return None
    
    print(f"Crop data for district {district_name} loaded successfully.")
    
    # Merge with nutrient dataset
    district_nutrients = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
    
    # Filter only the nutrient columns
    nutrient_totals = district_nutrients[nutrient_columns].sum()
    print(f"Total nutrients available in district {district_name}:")
    print(nutrient_totals)
    
    # Compare with RDI for each age group
    deficiencies = {}
    for age_group in age_nutrients_df['group']:
        # Get the RDI values for the current age group and filter relevant nutrient columns
        age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
        
        # Calculate deficiency
        deficiency = age_rdi.values[0] - nutrient_totals.values  # Subtraction is now aligned
        deficiency_dict = {col: val for col, val in zip(nutrient_columns, deficiency) if val > 0}  # Filter positive deficiencies
        deficiencies[age_group] = deficiency_dict
    
    # Display nutrient deficiencies for each age group
    for age_group, deficiency in deficiencies.items():
        print(f"\nNutrient deficiencies for {age_group}:")
        if deficiency:
            for nutrient, amount in deficiency.items():
                print(f"- {nutrient}: {amount:.2f}")
        else:
            print("No deficiencies.")
    
    return deficiencies

# Example usage
district_name = input("Enter the district name: ")
try:
    nutrient_deficiencies = identify_nutrient_deficiency(district_name, crops_df, nutrients_df, age_nutrients_df)
except KeyError as e:
    print(e)


Enter the district name: chitradurga
Crop data for district chitradurga loaded successfully.
Total nutrients available in district chitradurga:
carbohydrates     29695.0
protein            8174.3
fiber              4033.9
fat                9922.6
vitamin a        101234.0
vitamin c          4081.2
iron               2527.4
calcium             126.2
potassium        312725.0
magnesium         86609.0
dtype: float64

Nutrient deficiencies for adult male crossbred (1000 number):
- carbohydrates: 109470305.00
- protein: 20431825.70
- fiber: 10945966.10
- fat: 25540077.40
- vitamin a: 328398766.00
- vitamin c: 32845918.80
- iron: 2917472.60
- calcium: 364999873.80
- potassium: 1715187275.00
- magnesium: 145913391.00

Nutrient deficiencies for adult female crossbred (1000 number):
- carbohydrates: 109470305.00
- protein: 16831825.70
- fiber: 9745966.10
- fat: 25540077.40
- vitamin a: 255398766.00
- vitamin c: 27370918.80
- iron: 6567472.60
- calcium: 364999873.80
- potassium: 1715187275.00


In [21]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

# Function to project nutrient deficiencies using the same dataset
def project_deficiencies_ml_using_dataset(district_name, crops_df, nutrients_df, age_nutrients_df, future_years=10):
    # Standardize column names in age_nutrients_df
    age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip()

    # Nutrient columns to consider
    nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c', 
                        'iron', 'calcium', 'potassium', 'magnesium']
    
    # Filter data for the selected district
    district_data = crops_df[crops_df['district_name'] == district_name]
    if district_data.empty:
        print(f"No data found for the district: {district_name}")
        return None

    # Merge with nutrient dataset to get historical nutrient data
    district_nutrient_data = district_data.merge(nutrients_df, on='crop', how='inner')

    # Create yearly nutrient totals
    district_nutrient_data['year'] = district_nutrient_data['crop_year']  # Ensure 'year' column is present
    yearly_nutrient_totals = district_nutrient_data.groupby('year')[nutrient_columns].sum().reset_index()
    print(f"Yearly nutrient totals for district {district_name} (1997–2022):")
    print(yearly_nutrient_totals)

    # Initialize dictionary to store future projections
    future_projections = {}

    # Train regression model for each nutrient
    for nutrient in nutrient_columns:
        print(f"\nTraining model for {nutrient}...")
        
        # Extract year and nutrient data
        X = yearly_nutrient_totals['year'].values.reshape(-1, 1)  # Features (years)
        y = yearly_nutrient_totals[nutrient].values  # Target (nutrient totals)
        
        # Train Linear Regression model
        model = LinearRegression()
        model.fit(X, y)
        
        # Predict nutrient values for the next `future_years`
        future_years_range = np.arange(X.max() + 1, X.max() + 1 + future_years).reshape(-1, 1)
        future_values = model.predict(future_years_range)
        
        # Store projected values
        future_projections[nutrient] = future_values
        
        print(f"Projected {nutrient} values for the next {future_years} years:")
        for year, value in zip(future_years_range.flatten(), future_values):
            print(f"Year {year}: {value:.2f}")

    # Return projections
    return future_projections

# Example usage
district_name = input("Enter the district name: ")
future_projections = project_deficiencies_ml_using_dataset(district_name, crops_df, nutrients_df, age_nutrients_df)


Enter the district name: chitradurga
Yearly nutrient totals for district chitradurga (1997–2022):
    year  carbohydrates  protein  fiber    fat  vitamin a  vitamin c   iron  \
0   1997         1406.1    269.5  152.4  208.6       5147      171.9   89.2   
1   1998         1493.0    429.5  211.0  535.3       3067      156.2  127.8   
2   1999         1667.8    479.3  219.0  529.2       3199      157.7  147.4   
3   2000         1627.8    443.9  215.6  539.8       3199      120.5  133.0   
4   2001         1578.3    429.9  206.5  489.5       3137      116.8  127.7   
5   2002         1757.0    489.0  243.4  642.7       5452      406.7  145.2   
6   2003         1674.0    451.7  225.4  534.4       4555      350.8  137.0   
7   2004         1573.2    444.8  212.9  532.9       3252      156.7  134.3   
8   2005         1756.8    492.0  244.3  565.1       7777      287.9  152.2   
9   2006         1720.2    473.4  227.7  546.0       3344      161.7  139.2   
10  2007         1625.8    442.9 

In [22]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

# Function to recommend crops and project their growth
def recommend_crops_and_project_growth(deficiencies, nutrients_df, crops_df, district_name, future_years=5):
    # Filter crops rich in deficient nutrients
    recommended_crops = {}
    for age_group, deficiency in deficiencies.items():
        print(f"\nRecommendations for {age_group}:")
        recommended_crops[age_group] = {}
        for nutrient, deficiency_value in deficiency.items():
            # Find crops that are rich in the deficient nutrient
            nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
            recommended_crops[age_group][nutrient] = nutrient_rich_crops['crop'].tolist()
            print(f"- {nutrient}: {nutrient_rich_crops['crop'].tolist()}")
    
    # Project growth for recommended crops
    crop_growth_projections = {}
    for age_group, crops in recommended_crops.items():
        crop_growth_projections[age_group] = {}
        print(f"\nProjecting growth for crops recommended for {age_group}...")
        for nutrient, crop_list in crops.items():
            for crop in crop_list:
                # Filter historical data for the crop in the selected district
                crop_data = crops_df[(crops_df['crop'] == crop) & (crops_df['district_name'] == district_name)]
                if crop_data.empty:
                    continue
                
                yearly_production = crop_data.groupby('crop_year')['production'].sum().reset_index()
                X = yearly_production['crop_year'].values.reshape(-1, 1)
                y = yearly_production['production'].values
                
                # Train Linear Regression model
                model = LinearRegression()
                model.fit(X, y)
                
                # Predict future production for the next `future_years`
                future_years_range = np.arange(X.max() + 1, X.max() + 1 + future_years).reshape(-1, 1)
                future_production = model.predict(future_years_range)
                
                crop_growth_projections[age_group][crop] = future_production
                
                # Display projections for the crop
                print(f"Projected production for {crop} over the next {future_years} years:")
                for year, value in zip(future_years_range.flatten(), future_production):
                    print(f"- Year {year}: {value:.2f}")
    
    return recommended_crops, crop_growth_projections

# Example usage
district_name = input("Enter the district name: ")
recommended_crops, crop_growth_projections = recommend_crops_and_project_growth(
    deficiencies=nutrient_deficiencies,  # Pass deficiencies from previous calculations
    nutrients_df=nutrients_df,
    crops_df=crops_df,
    district_name=district_name
)


Enter the district name: chitradurag

Recommendations for adult male crossbred (1000 number):
- carbohydrates: ['other fibres', 'ginger', 'jute', 'bajra', 'barley', 'paddy', 'jowar', 'ragi', 'dry ginger', 'wheat', 'cardamom', 'korra', 'small millets', 'turmeric', 'samai', 'other cereals & millets', 'total foodgrain', 'varagu', 'black pepper', 'moong (green gram)', 'other kharif pulses', 'other rabi pulses', 'peas & beans (pulses)', 'arhar/tur', 'blackgram', 'pulses total', 'tea', 'coffee', 'cowpea (lobia)', 'lentil', 'masoor', 'other misc. pulses', 'urad', 'gram', 'dry chillies', 'horse-gram', 'moth', 'lab-lab', 'khesari', 'mesta', 'niger seed', 'guar seed', 'oilseeds total', 'rapeseed & mustard', 'tapioca', 'safflower', 'garlic', 'cashewnut', 'cashewnut processed', 'soyabean', 'linseed', 'rice', 'yam', 'cashewnut raw', 'sugarcane', 'arecanut', 'sesamum', 'arcanut (processed)', 'banana', 'jack fruit', 'atcanut (raw)', 'sapota', 'sweet potato', 'rubber', 'sunflower', 'maize', 'castor se


Projecting growth for crops recommended for adult female crossbred (1000 number)...

Projecting growth for crops recommended for young male crossbred (1000 number)...

Projecting growth for crops recommended for young female crossbred (1000 number)...

Projecting growth for crops recommended for adult male indigenous (1000 number)...

Projecting growth for crops recommended for adult female indigenous (1000 number)...

Projecting growth for crops recommended for young male indigenous (1000 number)...

Projecting growth for crops recommended for young female indigenous (1000 number)...


In [23]:
# Function to identify neighboring districts and suggest crop exchanges
def suggest_crop_exchanges(district_name, crops_df, nutrients_df, deficiencies, surplus_threshold=1.2):
    # Find neighboring districts (this example uses districts in the same state as neighbors)
    selected_district = crops_df[crops_df['district_name'] == district_name]
    if selected_district.empty:
        print(f"No data found for the district: {district_name}")
        return None

    state_name = selected_district['state_name'].iloc[0]
    neighboring_districts = crops_df[crops_df['state_name'] == state_name]['district_name'].unique().tolist()
    neighboring_districts.remove(district_name)  # Exclude the selected district itself

    print(f"Neighboring districts for {district_name}: {neighboring_districts}")

    # Initialize suggestions
    crop_exchanges = {}

    # Iterate through each age group's deficiencies
    for age_group, deficiency in deficiencies.items():
        print(f"\nEvaluating crop exchanges for {age_group}:")
        crop_exchanges[age_group] = {}
        
        for nutrient, deficit_value in deficiency.items():
            print(f"Deficient nutrient: {nutrient}")
            
            # Find neighboring districts with a surplus of this nutrient
            surplus_districts = []
            for neighbor in neighboring_districts:
                neighbor_data = crops_df[crops_df['district_name'] == neighbor].merge(
                    nutrients_df, on='crop', how='inner'
                )
                # Calculate total available nutrient in the neighbor district
                nutrient_total = neighbor_data[nutrient].sum()
                
                # Check if the nutrient is in surplus based on the threshold
                if nutrient_total > deficit_value * surplus_threshold:
                    surplus_districts.append((neighbor, nutrient_total))
            
            # Suggest crop exchanges for each surplus district
            if surplus_districts:
                crop_exchanges[age_group][nutrient] = []
                for surplus_district, total in surplus_districts:
                    surplus_crops = nutrients_df[
                        nutrients_df[nutrient] > 0
                    ].sort_values(by=nutrient, ascending=False)['crop'].tolist()
                    
                    crop_exchanges[age_group][nutrient].append({
                        'donor_district': surplus_district,
                        'available_amount': total,
                        'suggested_crops': surplus_crops
                    })
                    
                    print(f"- Donor District: {surplus_district}, Total {nutrient}: {total:.2f}")
                    print(f"  Suggested Crops: {surplus_crops}")
            else:
                print(f"No surplus found for {nutrient} in neighboring districts.")

    return crop_exchanges

# Example usage
district_name = input("Enter the district name: ")
crop_exchanges = suggest_crop_exchanges(
    district_name=district_name,
    crops_df=crops_df,
    nutrients_df=nutrients_df,
    deficiencies=nutrient_deficiencies  # Pass the previously calculated deficiencies
)


Enter the district name: chitradurga
Neighboring districts for chitradurga: ['belgaum', 'bellary', 'bidar', 'dharwad', 'hassan', 'kolar', 'mandya', 'mysore', 'raichur', 'tumkur']

Evaluating crop exchanges for adult male crossbred (1000 number):
Deficient nutrient: carbohydrates
No surplus found for carbohydrates in neighboring districts.
Deficient nutrient: protein
No surplus found for protein in neighboring districts.
Deficient nutrient: fiber
No surplus found for fiber in neighboring districts.
Deficient nutrient: fat
No surplus found for fat in neighboring districts.
Deficient nutrient: vitamin a
No surplus found for vitamin a in neighboring districts.
Deficient nutrient: vitamin c
No surplus found for vitamin c in neighboring districts.
Deficient nutrient: iron
No surplus found for iron in neighboring districts.
Deficient nutrient: calcium
No surplus found for calcium in neighboring districts.
Deficient nutrient: potassium
No surplus found for potassium in neighboring districts.
D